# Deep Learning study toy Project 1



백준 문제중에 Linear Algebra and Group 이라는 문제가 있다.

아주 단순한 문제로 "선형대수와 군"(이인석, 개정판)의 사진에 있는 모든 그림의 숫자를 합하면 되는 문제다.

자세한 문제는 [이곳을 참고하자](https://www.acmicpc.net/problem/15636)



> 문제를 확인했다면 ["선형대수와 군" 책의 사진을 보자](http://image.aladin.co.kr/product/5972/77/letslook/8952117441_f.jpg)

이 문제를 풀기 위해 고민하던 우리는 딥러닝 스터디에서 MNIST를 배운 기억을 떠올렸다. 

MNIST를 활용해서 문제를 풀어보자



> 참고로 지금부터 코드는 파이썬 3.6를 권장한다.<br>
> 만일 주피터 노트북의 파이썬이 3.6버전이 아니라면<br>
> conda install python=3.6 명령어를 이용하여 버전을 변경시키자

# Problem 1

### 이미지 가져오기

우리는 선형대수와 군 책을 가지고 있지않다. 따라서 이미지를 가져와야 하는데 마우스 오른쪽 클릭하여 이미지를 가져오기엔 체면이 살지 않는다. <br>
웹 스크래핑 실습때 배운 이미지 추출을 복습할겸 한번 가볍게 써보자

참고로 이미지를 가져올 주소는 http://image.aladin.co.kr/product/5972/77/letslook/8952117441_f.jpg 를 이용하면 된다.

이미지는 images 폴더 내부에 linear.jpg 라는 파일명으로 만들자.

##### 참고: urllib 함수는 파이썬 내장함수이다. 별로도 설치할 필요 없다.

In [1]:
from urllib.request import urlretrieve
import os

# 만일 images 폴더가 존재하지 않으면 해당 폴더를 만든다.
if not os.path.isdir('images'):
    os.mkdir('images')

# insert your code

# url으로 이미지 다운로드
url = "http://image.aladin.co.kr/product/5972/77/letslook/8952117441_f.jpg"
img_name = "linear"
urlretrieve(url, "./images/"+img_name+'.png')

('./images/linear.png', <http.client.HTTPMessage at 0x1e2749848d0>)

### 이미지 실행시키기

이미지가 제대로 가져와졌는지 실행시켜보자.<br>
이미지를 실행시키기 위해 PIL이라는 모듈을 사용하겠다. PIL은 이미지를 가공하기 좋은 모듈이다. 

##### 만일 아래 코드가 실행이 되지 않는다면 다음 명령어로 모듈을 설치하기 바란다.<br>
> conda install PIL

In [2]:
from PIL import Image
import PIL.ImageOps

# pil을 사용해 image open
img = Image.open("./images/linear.png")
img.show()

이미지가 정상적으로 나왔다면 성공적으로 가져온 것이다. 이제 모든 준비가 끝났다.

# Problem 2

### Image Cropping

#### EXAMPLE
이미지를 가져왔다면 이제 알맞은 크기로 잘라야 한다.<br>
우선 간단한 예제로 이미지를 잘라보자.

In [4]:
# 우선 이미지를 가져온다.
img = Image.open("./images/linear.png") 

# 자를 이미지의 좌상단과 우하단의 (x1, y2, x2, y2) 좌표로 bounding box를 만든다.
# 위치를 모를 경우 그림판이나 다른 툴을 이용하면 좌표를 알 수 있다.
# 아래 좌표는 예시이다.
area = (30, 34, 871, 745)

# 선택한 좌표를 가지고 crop 함수를 이용해 자른다.
cropped_img = img.crop(area)

#자른 이미지를 확인해본다.
cropped_img.show()

#자른 이미지를 저장한다.
path = './images/'
name = 'test'
cropped_img.save(path + name + '.png')

<br><br><br>



실행해보면 우리가 계산할 숫자들의 이미지가 나올 것 이다.<br>
하지만 우리가 원하는 사진은 숫자 하나 하나 잘라야 한다.

다음 사진을 보자
<img src="./pic/01.PNG" width=80%/>

### 우선 위 사진 처럼 사진을 하나씩 잘라보자. 파일의 형식은 0001.png 과 같은 형식이다.

#### 참고로 사진은 총  1450개 이다. 즉, 0001.png 부터 1450.png가 나와야 한다.

In [5]:
# insert your code
def get_cropped_image():
    img = Image.open("./images/linear.png") 
    count = 1
    # image cropping
    s_x, s_y, dx, dy = 30, 34, 16.84, 24.55
    for i in range(0,29):
        for j in range(0,50):
            cropped_img = img.crop((s_x+2, s_y+2, s_x+dx-2, s_y+dy-2))
            cropped_img.save(path + str(count).zfill(4) + '.png')
            s_x += dx
            count += 1
        s_x = 30
        s_y += dy
get_cropped_image()


실행 후 사진이 잘 나왔는지 확인해 보자.

# Problem 3

### image preprocessing

우리가 자른 이미지는 MNIST에 의해 훈련된 모델로 분류가 잘 될까?<br>
거의 분류가 되지 않을 것이다. 아니 그 전에 실행이 되지 않을것이다.

우리가 자른 이미지의 사이즈는(자르기에 따라 약간의 오차가 있겠지만)
 대략 (17, 24, 3)의 픽셀로 이루어져 있다. 각각 width, height, depth의 값이다.
 
하지만 MNIST의 데이터는 (28, 28, 1)로 사이즈가 서로 다르다.

또한 이미지 주변의 테두리도 잘라내고 싶다.

다음 순서로 이미지를 가공해보자.

> 1. 위와 마찬가지로 이미지를 일단 자른다.
2. 테두리의 검은 선을 자르기 위해 자른 이미지를 다시 한번 약간 자른다.
3. Image 모듈의 함수를 이용해서 28X28의 이미지로 scaling 시킨다.
4. depth를 1로 만들기 위해 사진을 흑백으로 변경시킨다. 이 때 convert 함수를 이용한다.
5. MNIST는 배경이 검은색 글씨가 흰색인 사진이다. 우리의 사진도 마찬가지로 만들어주기 위해 사진의 색을 반전시키자.

[모듈에 관한 자세한 설명은 이곳을 참고하자](https://pillow.readthedocs.io/en/3.1.x/reference/Image.html)



실행 결과는 다음과 같다<br><br>
<img src="./pic/02.PNG" width=80%/>


In [6]:
# insert your code
# 코드가 완성됐다면 함수로 만들어서 계속 사용 할 수 있게 하자
# 함수의 이름은 image_prep() 로 하 도록 하자.
def image_prep():
    count = 1
    basewidth,hsize = 28, 28
    s_x, s_y, dx, dy = 30, 34, 16.84, 24.55
    for i in range(0,29):
        for j in range(0,50):
            cropped_img = img.crop((s_x+2, s_y+2, s_x+dx-2, s_y+dy-2))
            cropped_img = cropped_img.resize((basewidth,hsize), Image.ANTIALIAS)
            cropped_img = cropped_img.convert('L')
            cropped_img = cropped_img.point(lambda x: 255 if x<100 else 0, '1')
            cropped_img.save(path + str(count).zfill(4) + '.png')
            s_x += dx
            count += 1
        s_x = 30
        s_y += dy
image_prep()



# Problem 4

### softmax로 MNIST 학습시키기

lab 7에서 혹은 이후 lab에서 실습한 예제를 이용해서 MNIST를 학습시키자.

In [33]:
# MNIST를 학습시켜보자. 복붙하지 말자
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

nb_classes = 10

# MNIST data image of shape 28*28 = 784
X = tf.placeholder(tf.float32, [None,784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None,nb_classes])

W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

# Hypothesis (using softmax)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b )

cost = tf.reduce_mean( -tf.reduce_sum(Y * tf.log(hypothesis),axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1 ).minimize(cost)

# Test Model
is_correct = tf.equal(tf.arg_max(hypothesis, 1),tf.arg_max(Y,1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# parameters
training_epochs = 100
batch_size = 100

with tf.Session() as sess:
    # Initalize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training Cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], feed_dict = {X: batch_xs, Y: batch_ys})
            avg_cost += c / total_batch
        print('Epoch: ', '%04d' % (epoch +1), 'cost = ', '{:9f}' .format(avg_cost))
    print(
        "Accuracy: ",
        accuracy.eval(
            session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}
        ),
    )
    saver = tf.train.Saver()
    save_path = saver.save(sess, "./mnist_softmax.ckpt")


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch:  0001 cost =   3.250581
Epoch:  0002 cost =   1.114488
Epoch:  0003 cost =   0.879867
Epoch:  0004 cost =   0.766728
Epoch:  0005 cost =   0.695723
Epoch:  0006 cost =   0.646931
Epoch:  0007 cost =   0.609382
Epoch:  0008 cost =   0.580029
Epoch:  0009 cost =   0.555805
Epoch:  0010 cost =   0.535441
Epoch:  0011 cost =   0.517672
Epoch:  0012 cost =   0.502294
Epoch:  0013 cost =   0.489347
Epoch:  0014 cost =   0.476816
Epoch:  0015 cost =   0.466283
Epoch:  0016 cost =   0.456361
Epoch:  0017 cost =   0.447424
Epoch:  0018 cost =   0.439284
Epoch:  0019 cost =   0.431450
Epoch:  0020 cost =   0.424835
Epoch:  0021 cost =   0.417885
Epoch:  0022 cost =   0.411885
Epoch:  0023 cost =   0.406362
Epoch:  0024 cost =   0.401231
Epoch:  0025 cost =   0.396031
Epoch:  0026 cost =   0.391290


간단한 모델을 이용하다 보니 정확도가 높지 않다.<br>

하지만 그대로 진행한다. 대략 88%이상의 정확도면 된다.

# Problem 5

### 이미지 데이터 가져오기

이미지의 리스트를 가져오기 위해서 glob 라는 함수를 사용하자

In [8]:
import glob

# 파일이 존재하는 폴더의 위치를 정해주고 
# 해당 위치의 모든 png file을 정렬해서 가져온다.
glob_path = glob.glob('./images/*.png')
# 이후 test_data list에 해당 이미지를 open하여 append한다.
test_data = []
for img_name in glob_path:
    im = Image.open(img_name).convert('L')
    test_data.append(im)

하나의 테스트 파일을 출력해 보자.
size가 28X28이면 알맞게 불러진 것이다.

In [9]:
print(test_data[0])
print(len(test_data))

<PIL.Image.Image image mode=L size=28x28 at 0x1E2753BFBE0>
1452


불러온 데이터를 하나 열어보자 



In [10]:
import matplotlib.pyplot as plt
plt.imshow(test_data[3],cmap="Greys",interpolation='nearest')
plt.show()

<Figure size 640x480 with 1 Axes>

# Problem 6

### 이미지 테스트 해보기

이미지를 테스트 해 보는 단계이다. 

훈련한 모델을 이용하여 모델이 제대로 분류를 하는지 확인해 보자.

확인을 하기 위해 사진이 들어있는 폴더 내에 0부터 9까지의 폴더를 만들어야 한다.

그리고 사진을 예측한 값의 이름을 가진 폴더에 넣게 된다.


> 폴더 조작을 위해 shutil 모듈을 이용해 보자

In [34]:
import numpy as np
import shutil

src = './images/'
dst = './images/'
# 폴더가 존재하지 않다면 폴더를 만들자
dir_init()
for i in range(10):
    if not os.path.isdir(src + str(i)):
        os.mkdir(src + str(i))
#dir_init()

# 학습한 모델을 가지고 예측을 해보자. 예측후 해당 폴더로 파일을 이동시킨다.


# placeholder y에 넣기 위해서 reshape
td_y = np.zeros((1450,10),dtype = "float32")

X = tf.placeholder(tf.float32, [None,784])
Y = tf.placeholder(tf.float32, [None,nb_classes])
W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b )

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess , "./mnist_softmax.ckpt")
    for i in range(1450):
        td_x = np.array(test_data[i], dtype = "float32").reshape(-1,784)
        # 0~1사이의 값으로 normalization
        td_x /= 255.0
        # Accuracy 측정
        predict = sess.run(hypothesis, feed_dict={X: td_x})
        # 10개의 데이터에 대한 결과값 보기
        #print(predict)
        #print(np.argmax(predict))
        shutil.copy(src+str(i+1).zfill(4)+'.png',dst+str(np.argmax(predict)))
        
        #fig = plt.figure()
        
        '''
        subplot = fig.add_subplot(2,5,i+1)
        # x,y축의 지점 표시 X
        subplot.set_xticks([])
        subplot.set_yticks([])
        # subplot의 제목을 i 번째 결과에 해당하는 숫자로 설정
        subplot.set_title('%d' %sess.run(tf.arg_max(predict,1)))
        subplot.imshow(td_x.reshape((28,28)),cmap="Greys",interpolation='nearest')
        plt.show()
        '''
        
    '''
    fig = plt.figure()
    for i in range(10):
        subplot = fig.add_subplot(2,5,i+1)
        # x,y축의 지점 표시 X
        subplot.set_xticks([])
        subplot.set_yticks([])
        
        # subplot의 제목을 i 번째 결과에 해당하는 숫자로 설정
        subplot.set_title('%d' %np.argmax(td_y[i:i+1]))
        subplot.imshow(td_x.reshape((28,28)),cmap=plt.cm.gray_r)
    plt.show()
    '''


INFO:tensorflow:Restoring parameters from ./mnist_softmax.ckpt


사진이 폴더에 들어가면 테스트에 성공한 것이다.

이제 폴더를 열어 결과를 확인해 보자.

결과가 잘 나왔으면 어떠한 기법을 사용한 것인지 잘 나오지 못하였다면 어떤 부분에서 문제가 생긴지 생각해 보자

개선 할 수 있다면 개선하여 도전해보자

# Problem 7

### 초기화 하기


테스트가 끝난 뒤 다시 테스트를 하려면 손이 많이 간다. 

이러한 점이 불편하게 느껴지니 자동으로 초기 상태로 돌려주는 프로그램을 만들고자 한다.

우선 아쉽게도 shutil 모듈 내부에는 모든 파일을 지우는 함수가 없다.

따라서 shutil의 모듈의 폴더를 지우는 함수를 이용하여 폴더를 지우고 해당 폴더를 다시 만드는 작업을 하자

그리고 위에서 만든 imag_prep() 함수를 이용하여 없어진 사진도 다시 복구하자

모두 다 끝난 뒤 init() 함수로 만들면 된다.


In [30]:
import os
import shutil

def dir_init():
    for i in range(10):
        if os.path.isdir(src + str(i)):
            shutil.rmtree(src + str(i))